In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from widgets import Chat, ChatInput, Message


In [2]:
# pip install altair vega_datasets
import altair as alt
from vega_datasets import data

source = data.barley()

In [3]:
chat = Chat()

Output()

In [4]:
chat_input = ChatInput()

ChatInputWidget()

In [20]:
import time
def responder(user_text: str):
    """Simulate response"""
    # simulate wainting for response
    time.sleep(2)
    # response chunks
    text = f"Echo: {user_text}"
    for ch in text:
        yield ch
        time.sleep(0.05)

In [21]:
def make_plot(output_msg):
    with output_msg:
        fig, ax = plt.subplots()
        ax.plot([1,2], [4,5])
        display(fig)
        plt.close(fig)

In [22]:
def make_df(output_msg):
    df = pd.DataFrame({"a": [1,2,3], "b": [4,5,6]})
    with output_msg:
        display(df)

In [23]:
def make_altair(output_msg):
    with output_msg:
        chart = (
            alt.Chart(source)
            .mark_bar()
            .encode(
                x='sum(yield):Q',
                y=alt.Y('variety:N', sort='-x'),
                color='site:N',
                tooltip=['variety','site','sum(yield)']
            )
            .properties(width=600, height=300, title='Barley Yield by Variety & Site')
        )
        display(chart) 

In [24]:
msg = chat_input.submitted.strip()

if msg:
    # User prompt
    user_msg = Message(role="user", emoji="👤")
    user_msg.set_message(markdown=msg)
    chat.add(user_msg)

    # AI response
    bot_msg = Message(role="assistant", emoji="🤖")
    # waiting for response ...
    bot_msg.set_gradient_text("Thinking hard 🤔")
    chat.add(bot_msg)
    if msg == "dataframe":
        bot_msg.clear() # clear thinking text
        bot_msg.set_message(markdown="DataFrame response created for you 🤔")
        make_df(bot_msg)
    elif msg == "plot":
        bot_msg.set_message(markdown="Plot response is below:")
        make_plot(bot_msg)
    elif msg == "altair":
        bot_msg.set_message(markdown="Altair response is below:")
        make_altair(bot_msg)
    else:
        # stream response
        for chunk in responder(msg):
            bot_msg.append_markdown(chunk)
    
    # clear the submitted prompt
    chat_input.submitted = ""